In [ ]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

## Dependencies

In [9]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pymongo
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from keras.utils import to_categorical
from config import mongo_conn

Using TensorFlow backend.


## Load Perfume Data

In [32]:
# Initialize PyMongo to work with MongoDBs
client = pymongo.MongoClient(mongo_conn)
db = client.perfume_db
perfume_df = pd.DataFrame(list(db.perfume_data.find()))
perfume_df.head()

,_id,brand,title,date,accords,rating_score,votes,longevity_poor,longevity_weak,longevity_moderate,...,notes_12,notes_13,notes_14,notes_15,notes_16,notes_17,notes_18,notes_19,notes_20,gender
0,603987b919085a1bd5bffcbe,The-Spirit-of-Dubai,Aamal The Spirit of Dubai for women and men,2017-01-01 00:00:00,"woody,earthy,animalic,amber,musky,balsamic",5.0,3,0,0,0,...,Base3Moss,Base4Agarwood (Oud),Base5Indian Oud,,,,,,,women
1,603987b919085a1bd5bffcbf,Ajmal,Aatifa Ajmal for women and men,2014-01-01 00:00:00,"fresh spicy,woody,musky,rose,amber",4.2,10,1,0,0,...,,,,,,,,,,women
2,603987b919085a1bd5bffcc0,Al-Jazeera-Perfumes,AA Al-Jazeera Perfumes for women and men,2000-01-01 05:00:00,"rose,woody,musky,oud,fruity",0.0,0,0,0,0,...,,,,,,,,,,women
3,603987b919085a1bd5bffcc1,Art-of-Scent-Swiss-Perfumes,aarewasser Art of Scent - Swiss Perfumes for w...,2010-01-01 00:00:00,"white floral,green,ozonic,fresh,animalic",0.0,1,0,0,0,...,,,,,,,,,,women
4,603987b919085a1bd5bffcc2,Hamidi-Oud-Perfumes,Aaliyah Hamidi Oud & Perfumes for women and men,2000-01-01 05:00:00,"woody,warm spicy,amber,balsamic,musky",0.0,2,0,0,0,...,,,,,,,,,,women


### Preprocess Data

In [41]:
# perfume_df['accords'] = perfume_df['accords'].tolist()
# perfume_df.head()
# df2 = pd.DataFrame(perfume_df.accords.values.tolist())
# df2 = perfume_df.accords.apply(pd.Series)
# df2


In [39]:
# Reformat data

perfume_values = perfume_df.values
X = perfume_values[:, 7:36]
y = perfume_values[:, 5]
print(X.shape, y.shape)
X

(51212, 29) (51212,)


array([[0, 0, 0, ..., '', '', ''],
       [1, 0, 0, ..., '', '', ''],
       [0, 0, 0, ..., '', '', ''],
       ...,
       [0, 0, 0, ..., '', '', ''],
       [0, 0, 0, ..., '', '', ''],
       [1, 0, 0, ..., '', '', '']], dtype=object)

### Dummy Encoding (Binary Encoded Data)

In [40]:
data = X.copy()

data_binary_encoded = pd.get_dummies(data)
data_binary_encoded.head()

Exception: Data must be 1-dimensional

In [13]:
# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
# Create a StandardScater model and fit it to the training data

X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

In [ ]:
# Transform the training and testing data using the X_scaler and y_scaler models

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
# Create a LinearRegression model and fit it to the scaled training data

model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)


In [ ]:
# Used X_test_scaled, y_test_scaled, and model.predict(X_test_scaled) to calculate MSE and R2

from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = model.score(X_test_scaled, y_test_scaled)

print(f"MSE: {MSE}, R2: {r2}")